In [1]:
import os

In [2]:
mapping = {
    "Bicycle": [1],
    "Boat": [8],
    "Bottle": [39],
    "Bus": [5, 7],
    "Car": [2],
    "Cat": [15],
    "Chair": [13, 56],
    "Cup": [40, 41],
    "Dog": [16],
    "Motorbike": [3],
    "People": [0],
    "Table": [60],
}

In [3]:
precisions = []
recalls = []
ious = []

In [4]:
import math

def xywh_to_xyxy(x, y, w, h):
    x1 = x - (w / 2)
    y1 = y - (h / 2)
    
    x2 = x + (w / 2)
    y2 = y + (h / 2)
    
    return (int(x1), int(y1)), (int(x2), int(y2))


def xyxy_to_xywh(c1, c2):
    w = c2[0] - c1[0]
    h = c2[1] - c1[1]
    x = c2[0] + c1[0] / 2
    y = c2[1] + c1[1] / 2

    return x, y, w, h

def hasOverlap(c1, c2, c3, c4):
    x = 0
    y = 1
    
    intercept_x = (c1[x] < c3[x] and c3[x] < c2[x]) or (c1[x] < c4[x] and c4[x] < c2[x])
    intercept_y = (c1[y] < c3[y] and c3[y] < c2[y]) or (c1[y] < c4[y] and c4[y] < c2[y])

def calculateIoU(box1, box2):
    x = 0
    y = 1
    
    c1, c2 = xywh_to_xyxy(*box1)
    c3, c4 = xywh_to_xyxy(*box2)
    
    area1 = (c2[x] - c1[x]) * (c2[y] - c1[y])
    area2 = (c4[x] - c3[x]) * (c4[y] - c3[y])
    
    x_dist = min(c2[x], c4[x]) - max(c1[x], c3[x])
    y_dist = min(c2[y], c4[y]) - max(c1[y], c3[y])
    
    areaOverlap = 0
    
    if x_dist > 0 and y_dist > 0: 
        areaOverlap = x_dist * y_dist
    
    # print(f"c1: {c1} | c2: {c2} | c3: {c3} | c4: {c4} |")
    # print(f"Area1: {area1} | Area2: {area2} | Area Overlap: {areaOverlap}")
    
    return areaOverlap / (area1 + area2 - areaOverlap)
    

In [5]:
iou_threshold = 0.75
target = "ZeroDCE"
source_path = "./datasets/test/annotation/exp0/"
sample_path = f"./datasets/test/annotation/{target}/"

output = open(f'./datasets/test/annotation/Results/{target}-{iou_threshold}.txt', 'w')

for root, _, files in os.walk(os.path.join(source_path, 'labels')):
    
    for file in files:
        target_cxywh = []
        sample_cxywh = []
        
        TP = 0
        OU = 0
                
        try:
            with open(os.path.join(source_path, 'labels', file)) as annotation:
                target_cxywh = [[mapping[line.split()[0]]] + list(map(float, line.split()[1:5])) for line in annotation.readlines()]
                
            with open(os.path.join(sample_path, 'labels', file)) as annotation:
                sample_cxywh = [[int(line.split()[0])] + list(map(lambda x : float(x) * 256, line.split()[1:5])) for line in annotation.readlines()]
                
            if len(target_cxywh)  == 0 or len(sample_cxywh) == 0: continue
        except:
            continue
        
        for target in target_cxywh:
            
            for sample in sample_cxywh:
                # Finding the IOU
                iou = calculateIoU(target[1:5], sample[1:5])
                
                # print(f"Target: {target[0]} | Sample: {sample[0]} | IoU: {iou}")
                if (iou > iou_threshold):
                    # Make correct prediction
                    TP += 1
                    OU += iou
                    break
                
        precision = TP / len(sample_cxywh)
        recall = TP / len(target_cxywh)
        
        output.write("File: {} | TP: {} | Total Positives: {:02d} | Total TP: {:02d} | Precision: {:.6f} | Recall: {:.6f}\n".format(file, TP, len(sample_cxywh), len(target_cxywh), precision, recall))
        precisions.append(precision)
        recalls.append(recall)
        if (TP != 0): ious.append(OU / TP)
        
output.close()
        

In [6]:
print("mAP: %.6f" % (sum(precisions) / len(precisions)))
print("Recall: %.6f"  % (sum(recalls) / len(recalls)))
print("IoU: %.6f"  % (sum(ious) / len(ious)))

mAP: 0.403282
Recall: 0.515481
IoU: 0.865735
